### Задача 1:

Для функции f(x) найти точку локального минимума, пользуясь
указанными методами. Сравнить эффективность методов (например, количество
вычислений функции или производных для достижения заданной точности). Если
локальных минимумов несколько, достаточно найти только один из них. Первоначальный
отрезок локализации минимума можно находить с помощью метода равномерного поиска
или графического метода. Для контроля правильности реализации методов, используйте
встроенные функции из библиотеки scipy.

### Решение на языке Python:

In [19]:
from scipy.optimize import minimize_scalar 

def quadratic_interpolation(f, x1, delta_x, eps1, eps2):

  # Шаг 2
  x2 = x1 + delta_x

  # Шаг 3
  f1 = f(x1)
  f2 = f(x2)

  while True:
    # Шаг 4
    if f1 > f2:
      x3 = x1 + 2 * delta_x
    else:
      x3 = x1 - delta_x

    # Шаг 5
    f3 = f(x3)

    # Шаг 6
    F_min = min(f1, f2, f3)
    x_min = x1 if f1 == F_min else x2 if f2 == F_min else x3

    # Шаг 7
    try:
      x = 0.5 * ((x2**2 - x3**2) * f1 + (x3**2 - x1**2) * f2 + (x1**2 - x2**2) * f3) / ((x2 - x3) * f1 + (x3 - x1) * f2 + (x1 - x2) * f3)
    except ZeroDivisionError:
      # Знаменатель в формуле x обращается в нуль
      x1 = x_min
      x2 = x1 + delta_x
      f1 = f(x1)
      f2 = f(x2)
      continue

    # Шаг 8
    if abs((F_min - f(x)) / f(x)) < eps1 and abs((x_min - x) / x) < eps2:
      # Условия окончания выполнены
      return x, F_min

    # Если хотя бы одно из условий не выполнено
    if x_min <= x <= x3:
      # Выбираем наилучшую точку и две точки по обе стороны от нее
      x1, x2, x3 = sorted((x_min, x, x3))
    else:
      # x не принадлежит [x1, x3]
      x1 = x
      x2 = x1 + delta_x
      f1 = f(x1)
      f2 = f(x2)


# Определяем целевую функцию
def f(x):
    return 3*x**4 - 8*x**3 + 6*x**2

# Пример использования метода квадратичной интерполяции
x_min = quadratic_interpolation(f, -10, 0.1, 1e-8, 1e-8)
print("Локальный минимум:", x_min[0])
print("Значение функции в минимуме:", x_min[1])

result = minimize_scalar(f)
print("Локальный минимум (scipy):", result.x)
print("Значение функции в минимуме (scipy):", result.fun)

Локальный минимум: 0.006688962196245331
Значение функции в минимуме: 0.00026606506525106156
Локальный минимум (scipy): 0.0
Значение функции в минимуме (scipy): 0.0


In [24]:
import scipy.optimize as opt
import numpy as np
from scipy.optimize import approx_fprime, minimize


def newton_method(f, grad_f, hessian_f, x0, eps1, eps2, M):
  """
  Нахождение точки локального минимума функции f(x) методом Ньютона.

  Args:
    f: Функция, для которой ищется минимум.
    grad_f: Функция, вычисляющая градиент функции f.
    hessian_f: Функция, вычисляющая матрицу Гессе функции f.
    x0: Начальная точка.
    eps1: Точность по градиенту.
    eps2: Точность по координате.
    M: Предельное число итераций.

  Returns:
    Кортеж (x, f_min), где x - точка минимума, f_min - значение функции в этой точке.
  """

  # Шаг 2

  k = 0

  while True:
    # Шаг 3

    grad_f_x_k = grad_f(x0)

    # Шаг 4

    if np.linalg.norm(grad_f_x_k) <= eps1:
      # Критерий остановки по градиенту
      return x0, f(x0)

    # Шаг 5

    if k >= M:
      # Предельное число итераций
      return x0, f(x0)

    # Шаг 6

    H_x_k = hessian_f(x0)

    # Шаг 7

    try:
      H_inv_x_k = np.linalg.inv(H_x_k)
    except np.linalg.LinAlgError:
      # Обратная матрица не существует
      return x0, f(x0)

    # Шаг 8

    if np.all(H_inv_x_k > 0):
      # Матрица Гессе положительно определена
      d_k = -np.dot(H_inv_x_k, grad_f_x_k)
    else:
      # Матрица Гессе не положительно определена
      d_k = -grad_f_x_k

    # Шаг 9

    # Шаг 10

    if d_k.ndim == 1:
      # Вектор
      t_k = 1
    else:
      # Матрица
      t_k = armijo_rule(f, x0, d_k)

    x_k_next = x0 + t_k * d_k

    # Шаг 11

    if abs(np.linalg.norm(x_k_next - x0)) <= eps2 and abs(f(x_k_next) - f(x0)) <= eps2:
      # Критерий остановки по координатам
      return x_k_next, f(x_k_next)

    # Переход к следующей итерации

    x0 = x_k_next
    k += 1

def f(x):
    return 3*x**4 - 8*x**3 + 6*x**2

def grad_f(x):
  return opt.approx_fprime(x, f, epsilon=1e-6)

def hessian_f(x):
  return opt.hessian(f, x, epsilon=1e-6)


x, f_min = newton_method(f, grad_f, hessian_f, 0, 1e-6, 1e-6, 100)

print(f"x = {x:.4f}")
print(f"f(x) = {f_min:.4f}")


AttributeError: module 'scipy.optimize' has no attribute 'hessian'